#### RNN Model Hakkında

`tensorflow`: TensorFlow kütüphanesi, makine öğrenimi ve derin öğrenme projelerinde kullanılan bir açık kaynaklı yazılım kütüphanesidir.  

`tensorflow.keras.models.Sequential`: Bir Keras modelinin en basit tipidir. Sıralı bir model, katmanları ardışık bir şekilde sıralar.  

`tensorflow.keras.layers.Dense`: Tam bağlı (fully connected) bir katmandır. Tüm girdi nöronları tüm çıktı nöronları ile bağlıdır.  

`tensorflow.keras.layers.Dropout`: Aşırı uyuma karşı ağın dayanıklılığını artırmak için kullanılan bir katmandır. Rastgele belirtilen bir oranda nöronları devre dışı bırakarak overfitting'i azaltır.  

`tensorflow.keras.layers.LSTM`: Uzun-kısa vadeli bellek (LSTM), RNN'lerde aşırı uyumu azaltmak için kullanılan bir özel bir türdür. LSTM katmanları, önceki girdilerden bilgi saklayarak, modelin uzun vadeli bağımlılıkları öğrenmesine yardımcı olur.
tensorflow.keras.layers.BatchNormalization: Verileri normalize ederek, ağın istikrarını artıran bir katmandır. Bu katmanın çıkışları, ağdaki diğer katmanlara daha uygun bir şekilde beslenir.  

`tensorflow.keras.optimizers.Adam`: Gradient descent optimizasyonu için bir yöntemdir. Optimizasyonun hızını artırmak için momentum terimi kullanır ve ayrıca adaptif öğrenme oranları kullanır.  

`tensorflow.keras.callbacks.EarlyStopping`: Ağın aşırı uyumu (overfitting) durumunda eğitimi durdurmak için kullanılan bir geri arama işlevi (callback).  

`tensorflow.keras.callbacks.ReduceLROnPlateau`: Öğrenme oranını azaltmak için kullanılan bir geri arama işlevi. Eğitim sırasında doğruluk (val_accuracy) değeri belirli bir sayıda epoch boyunca artmazsa öğrenme oranını azaltır.  

`tensorflow.keras.callbacks.ModelCheckpoint`: Eğitim sırasında ağın en iyi performansını kaydetmek için kullanılan bir geri arama işlevi. Epoch bazında modeli kaydeder ve doğruluk değeri en iyisiyse dosyaya kaydeder.  

`tensorflow.keras.losses.Huber`: Huber kaybı, aşırı uyuma karşı dayanıklılığı artırmak için kullanılan bir kayıp fonksiyonudur. Bu kayıp fonksiyonu, özellikle verilerde aykırı değerler olduğunda daha az hassastır.  

`tensorflow.keras.metrics.RootMeanSquaredError`: Ortalama karesel hatayı ölçmek için kullanılan bir metrik fonksiyondur.  

#### Notlar:
* Zaten scale edilmiş verilerde Batch Normalization kullanmak ne derece mantıklı?  
BatchNormalization'ın katmanlarda kullanılması faydalı olabilir. Bu katmanlarda, BatchNormalization ağırlıkların ve bias'ların dağılımını kontrol ederek daha istikrarlı ve hızlı bir eğitim süreci sağlayabilir.  

* Pek çok callbacks çağırmamızın nedeni, farklı amaçlar için farklı callbacks kullanarak eğitim sürecini yönetmek ve modelin performansını artırmak istememizdir. Örneğin, EarlyStopping callback'i, modelin eğitim sırasında overfitting yapmasını engellemek için kullanılabilirken, ModelCheckpoint callback'i, en iyi modeli kaydetmek için kullanılabilir. ReduceLROnPlateau callback'i ise, modelin eğitim sırasında learning rate'i otomatik olarak ayarlayarak daha hızlı bir şekilde eğitilmesini sağlayabilir.

<hr>

In [3]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
df = pd.read_csv('../Datasets/Manipulated Data/train.csv')

df = df[['period','target']]
df['period'] = pd.to_datetime(df['period'])
df.set_index('period', inplace=True)

df.head(3),df.tail(3),len(df)

(            target
 period            
 2010-10-15   10639
 2010-10-22   10036
 2010-10-29    9886,
             target
 period            
 2020-03-06   10941
 2020-03-13    9176
 2020-03-20    6444,
 493)

Dataset Splitting

In [5]:
train = df.loc[:'2019-03-20',:]
test = df.loc['2019-03-21':,:]

Scaling

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [7]:
scaler.fit(train)

scaled_train = scaler.transform(train)
scaled_test = scaler.transform(test)

In [8]:
scaled_train[:5]

array([[0.87372014],
       [0.71541087],
       [0.67603045],
       [0.62877396],
       [0.51850879]])

Time Step Example

In [9]:
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 3 # Kaç adet geriye bakacağımızı belirliyoruz.
n_features = 1 # Veri setimizdeki tahmin edilecek değer sayısı

generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

In [10]:
X,y = generator[0]
print(f'Given the Array: \n{X.flatten()}')
print(f'Predict this y: \n {y}')

Given the Array: 
[0.87372014 0.71541087 0.67603045]
Predict this y: 
 [[0.62877396]]


In [11]:
X.shape

(1, 3, 1)

Real Time Step

In [12]:
n_input = 52 # 52 hafta geriye bakıp tahmin yapacağız. 
n_features = 1 # Veri setimizdeki tahmin edilecek değer sayısı

train_generator = TimeseriesGenerator(scaled_train, scaled_train, length=n_input, batch_size=1)

Model Building

In [40]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [42]:
model = Sequential()

model.add(LSTM(128, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128)               66560     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


In [44]:
import time
start_time = time.time()

model.fit(train_generator,epochs=25, verbose=0)

print("--- %s seconds ---" % (time.time() - start_time))

--- 155.02551412582397 seconds ---


In [45]:
fig = px.line(model.history.history, y=['loss'])
fig.show()

In [46]:
last_train_batch = scaled_train[-52:]
last_train_batch = last_train_batch.reshape((1, n_input, n_features))
last_train_batch.shape

(1, 52, 1)

In [47]:
model.predict(last_train_batch)

1/1 [==============================] - 0s 296ms/step


array([[0.7517976]], dtype=float32)

In [48]:
scaled_test[0]

array([0.75794172])

In [53]:
def lstm_prediction(model, scaled_train, n_input, test):
    import warnings
    warnings.filterwarnings("ignore")
    
    test_predictions = []

    first_eval_batch = scaled_train[-n_input:]
    current_batch = first_eval_batch.reshape((1, n_input, n_features))

    for i in range(len(test)):
        
        # get the prediction value for the first batch
        current_pred = model.predict(current_batch, verbose=0)[0]
        
        # append the prediction into the array
        test_predictions.append(current_pred) 
        
        # use the prediction to update the batch and remove the first value
        current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
        
    true_predictions = scaler.inverse_transform(test_predictions)
    test['predictions'] = true_predictions
    
    return test

In [54]:
test = lstm_prediction(model, scaled_train, n_input, test); test.head()

,target,predictions
period,,
2019-03-22,10198,10174.597121
2019-03-29,11328,10244.697303
2019-04-05,10475,10307.210913
2019-04-12,10444,10371.114426
2019-04-19,9878,10436.407841


In [55]:
fig = px.line(test, y=['target','predictions'])
fig.show()

Metrics

In [56]:
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
mse = mean_squared_error(test['target'], test['predictions'])
mape = mean_absolute_percentage_error(test['target'], test['predictions'])
rmse = np.sqrt(mse)
print(f'MSE: {mse}\nRMSE: {rmse}\nMAPE: {mape}')

MSE: 787851.8706472607
RMSE: 887.6102019734004
MAPE: 0.07296010407294226


<hr>